In [133]:
import os
import pandas as pd
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np
import copy 

df = pd.read_csv("../feature_extraction/iclr.csv")

df = df.sample(frac=1)

In [134]:
x = copy.deepcopy(df)[["num_equations","mean_num_new_symbols_introduced","num_overall_unique_symbols","mean_num_unique_symbols","std_of_unique_symbols","max_representational_complexity"]]
y = copy.deepcopy(df).recommendation_avg

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3,
                                                    random_state=0
                                                    )

linreg = LinearRegression()
linreg.fit(X_train, y_train)

def norm(x,y):
    return np.mean((np.asarray(x)-np.asarray(y))**2)

print("Test error Linear Regression: {0}".format(norm(linreg.predict(X_test),y_test)))
print("Test error Avg: {0}".format(norm(np.mean(y_train),y_test)))
print("-------------------------------------------------------------------------")
cdf = pd.DataFrame(linreg.coef_, X_train.columns, columns=['Coefficients'])
print(cdf)

Test error Linear Regression: 1.9068342468064563
Test error Avg: 1.9272441849563053
-------------------------------------------------------------------------
                                 Coefficients
num_equations                        0.000091
mean_num_new_symbols_introduced     -0.453233
num_overall_unique_symbols           0.004151
mean_num_unique_symbols             -0.025036
std_of_unique_symbols                0.071345
max_representational_complexity     -0.013087


In [136]:

x = copy.deepcopy(df)[["num_equations","mean_num_new_symbols_introduced","num_overall_unique_symbols","mean_num_unique_symbols","std_of_unique_symbols","max_representational_complexity"]]
y = copy.deepcopy(df).status
y = np.asarray(y)
y[y == "Withdraw"] = 0
y[y == "Reject"] = 1
y[y == "Desk Reject"] = 2
y[y == "Top-25%"] = 3
y[y == "Top-5%"] = 4
y[y == "Poster"] = 5



X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3,
                                                    random_state=0
                                                    )
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

logreg = LogisticRegression(max_iter=50000)
logreg.fit(X_train, y_train)

def acc(x,y):
    return np.mean((np.asarray(x) == np.asarray(y)).astype(np.int32))

print("Train error Linear Regression: {0}".format(acc(logreg.predict(X_train),y_train)))
print("Test error Linear Regression: {0}".format(acc(logreg.predict(X_test),y_test)))


print("Test error guess: {0}".format(acc(np.random.choice(y_train,size=y_test.size),y_test)))

print("-------------------------------------------------------------------------")
print(logreg.coef_.shape)

clf = pd.DataFrame(columns=["equations","mean_num_nsi","num_unique","mean_num_unique","std_unique","complexity"],data=logreg.coef_)
clf["status"] = ["Withdraw","Reject","Desk Reject","Top-25%","Top-5%","Poster"]
clf = clf[["status","equations","mean_num_nsi","num_unique","mean_num_unique","std_unique","complexity"]]
pd.set_option('display.width',1000)
print(clf)


Train error Linear Regression: 0.4809384164222874
Test error Linear Regression: 0.4641025641025641
Test error guess: 0.3264957264957265
-------------------------------------------------------------------------
(6, 6)
        status  equations  mean_num_nsi  num_unique  mean_num_unique  std_unique  complexity
0     Withdraw  -0.000344      1.147442    0.001930         0.011160   -0.146638    0.016122
1       Reject  -0.000226      0.474874    0.002974         0.060909   -0.139321    0.008381
2  Desk Reject   0.000318     -0.709398   -0.015740        -0.114151    0.034875    0.057034
3      Top-25%  -0.000204     -0.433036    0.007831         0.050299   -0.025749   -0.025652
4       Top-5%   0.000522     -0.190670   -0.003895        -0.003767    0.157170   -0.028215
5       Poster  -0.000066     -0.289213    0.006901        -0.004451    0.119664   -0.027671


/home/miri/miniconda3/envs/DataLit/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF F,G EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
